# Josh's Haiti 2010 SAM Project

In [1]:
import numpy as np
import pandas as pd
import glob
import xml.etree.ElementTree as ET

In [5]:
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                    int(root.find('size')[0].text),
                    int(root.find('size')[1].text),
                    member[0].text,
                    int(member[4][0].text),
                    int(member[4][1].text),
                    int(member[4][2].text),
                    int(member[4][3].text)
                    )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [25]:
train_data = xml_to_csv('./train/annotations')

In [26]:
train_data

,filename,width,height,class,xmin,ymin,xmax,ymax


In [6]:
import csv
import xml.etree.cElementTree as ET

def csv_to_xml(csv_path, resized_images_path, labels_path, folder):
    f = open(csv_path, 'r')
    reader = csv.reader(f)
    header = next(reader)
    old_filename = None
    for row in reader:
        filename = row[0]
        if filename == old_filename:
            object = ET.SubElement(annotation, 'object')
            ET.SubElement(object, 'name').text = row[3]
            ET.SubElement(object, 'pose').text = 'Unspecified'
            ET.SubElement(object, 'truncated').text = '0'
            ET.SubElement(object, 'difficult').text = '0'
            bndbox = ET.SubElement(object, 'bndbox')
            ET.SubElement(bndbox, 'xmin').text = row[4]
            ET.SubElement(bndbox, 'ymin').text = row[5]
            ET.SubElement(bndbox, 'xmax').text = row[6]
            ET.SubElement(bndbox, 'ymax').text = row[7]
        else:
            if old_filename is not None:
                labels_file = old_filename.replace('.jpg', '.xml')
                tree = ET.ElementTree(annotation)
                tree.write(labels_path + labels_file)

            annotation = ET.Element('annotation')
            ET.SubElement(annotation, 'folder').text = folder
            ET.SubElement(annotation, 'filename').text = filename
            ET.SubElement(annotation, 'path').text =     resized_images_path + filename
            source = ET.SubElement(annotation, 'source')
            ET.SubElement(source, 'database').text = 'Unknown'
            size = ET.SubElement(annotation, 'size')
            ET.SubElement(size, 'width').text = row[1]
            ET.SubElement(size, 'height').text = row[2]
            ET.SubElement(size, 'depth').text = '3'
            ET.SubElement(annotation, 'segmented').text = '0'

            object = ET.SubElement(annotation, 'object')
            ET.SubElement(object, 'name').text = row[3]
            ET.SubElement(object, 'pose').text = 'Unspecified'
            ET.SubElement(object, 'truncated').text = '0'
            ET.SubElement(object, 'difficult').text = '0'
            bndbox = ET.SubElement(object, 'bndbox')
            ET.SubElement(bndbox, 'xmin').text = row[4]
            ET.SubElement(bndbox, 'ymin').text = row[5]
            ET.SubElement(bndbox, 'xmax').text = row[6]
            ET.SubElement(bndbox, 'ymax').text = row[7]
        old_filename = filename
    f.close()

In [7]:
from imgaug.augmentables.bbs import BoundingBoxesOnImage
from imgaug import augmenters as iaa

aug = iaa.SomeOf(2, [
    iaa.Affine(scale=(0.5, 1.5)),
    iaa.Affine(rotate=(-60, 60)),
    iaa.Affine(translate_percent={"x": (-0.3, 0.3), "y": (-0.3,   0.3)}),
    iaa.Fliplr(1),
    iaa.Multiply((0.5, 1.5)),
    iaa.GaussianBlur(sigma=(1.0, 3.0)),
    iaa.AdditiveGaussianNoise(scale=(0.03 * 255, 0.05 * 255)),
    iaa.Add((-25, 25)),
    iaa.MotionBlur(k=15),
    iaa.MultiplySaturation((0.5, 1.5)),
    iaa.LogContrast(gain=(0.6, 1.4)),
    iaa.Flipud(1)
])

In [27]:
import imgaug as ia
ia.seed(1)
# imgaug uses matplotlib backend for displaying images
%matplotlib inline
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
from imgaug import augmenters as iaa 
# imageio library will be used for image input/output
import imageio
import pandas as pd
import numpy as np
import re
import os
import glob
# this library is needed to read XML files for converting it into CSV
import xml.etree.ElementTree as ET
import shutil

In [9]:
# function to convert BoundingBoxesOnImage object into DataFrame
def bbs_obj_to_df(bbs_object):
#     convert BoundingBoxesOnImage object into array
    bbs_array = bbs_object.to_xyxy_array()
#     convert array into a DataFrame ['xmin', 'ymin', 'xmax', 'ymax'] columns
    df_bbs = pd.DataFrame(bbs_array, columns=['xmin', 'ymin', 'xmax', 'ymax'])
    return df_bbs

In [16]:

if __name__ == '__main__':
    xml_df = xml_to_csv('labels_path/')
    for i in range(10):
        augmented_images_df = ia(xml_df, 'resized_images/', 'resized_images/', 'aug{}_'.format(i), aug)
        augmented_images_df.to_csv('aug{}_images.csv'.format(i), index=False)
        csv_to_xml(csv_path='aug{}_images.csv'.format(i),
                   resized_images_path='resized_images/', 
                   labels_path='labels_path/',
                   folder='resized_images')
        os.remove('aug{}_images.csv'.format(i))

TypeError: 'module' object is not callable

In [28]:
import os
import shutil
import random

images_path = 'resized_images/'
labels_path = 'labels/'
train_path = 'tents/train/'
validation_path = 'dataset/validation/'


for image_file in os.listdir(images_path):
    labels_file = image_file.replace('.jpg', '.xml')
    if random.uniform(0, 1) > 0.2:
        shutil.copy(images_path + image_file, train_path + 'images/'
                    + image_file)
        shutil.copy(labels_path + labels_file, train_path +
                    'annotations/' + labels_file)
    else:
        shutil.copy(images_path + image_file, validation_path +
                    'images/' + image_file)
        shutil.copy(labels_path + labels_file, validation_path +
                    'annotations/' + labels_file)

FileNotFoundError: [Errno 2] No such file or directory: 'resized_images/'

In [ ]:
plt.figure(figsize = (10,6))
plt.title('Count of photos in Train, Validation, and Test Sets',  fontsize=20)
plt.barh(range(len(photos_dict)), list(photos_dict.values()), align='center')
plt.yticks(range(len(photos_dict)), list(photos_dict.keys()), rotation= 0)
plt.xlabel('Count', fontsize=18)
plt.ylabel('Photos', fontsize=18)
# # for python 2.x:
# plt.bar(range(len(D)), D.values(), align='center')  # python 2.x
# plt.xticks(range(len(D)), D.keys())  # in python 2.x

plt.show()